# CASO FINAL: ANÁLISIS DE PORTAFOLIO 
####
### Nombres: Marco Ocampo y Javier Hernández
### Curso: Python for Finance
### Fecha: 22 de noviembre del 2023

In [2]:
### Cargamos las respectivas librerías para llevar a cabo el análisis de portafolios

import numpy as np
import plotly.express as px
import yfinance as yf
import datetime
import pyfolio as pf

import scipy.stats as stats
import matplotlib.pyplot as plt
import math
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.efficient_frontier import EfficientFrontier

In [7]:
import datetime

In [4]:
### Cargamos las librerías para hacer los dashboards

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from datetime import datetime
import dash

from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc

In [5]:
pip install PyPortfolioOpt

Note: you may need to restart the kernel to use updated packages.


## PARTE 1:

### Empresas del sector consumo masivo

#### Nestle [NSRGY]:

Sub-Industria: Alimentación y bebidas, cuidado de la salud.
Descripción: Nestlé es una multinacional suiza de alimentos y bebidas. Es conocida por su amplia variedad de productos que abarcan desde alimentos infantiles y chocolates hasta café, productos lácteos, alimentos para mascotas y productos de cuidado de la salud. Nestlé es una de las mayores empresas de alimentos y bebidas del mundo.

#### Walmart [WMT]:

Sub-Industria: Retail, supermercados.
Descripción: Walmart es la cadena minorista más grande del mundo. Fundada en Estados Unidos, opera tiendas minoristas bajo diferentes formatos, que incluyen supercentros, tiendas de descuento y clubes de almacenes. Walmart ofrece una amplia variedad de productos, desde comestibles y ropa hasta electrónicos y artículos para el hogar. Además de sus operaciones en tiendas físicas, Walmart ha expandido su presencia en el comercio electrónico.

#### Johnson & Johnson [JNJ]:|

Sub-Industria: Cuidado de la salud, productos farmacéuticos, productos de consumo.
Descripción: Johnson & Johnson es una empresa multinacional estadounidense que se especializa en productos relacionados con la salud y el bienestar. Opera en tres segmentos principales: productos farmacéuticos, dispositivos médicos y productos de consumo. La compañía es conocida por marcas emblemáticas como Band-Aid, Tylenol, Neutrogena y sus contribuciones a la investigación y desarrollo de medicamentos y dispositivos médicos. Además de su enfoque en la salud, Johnson & Johnson también tiene divisiones que se centran en la investigación y desarrollo de tecnologías médicas avanzadas.

### Empresas del sector de Producción industrial

#### Siemens AG [SIEGY]

Sub-Industria: Tecnología Industrial / Equipos y Maquinaria Industriales.
Descripción: Siemens AG es una empresa alemana líder en tecnología industrial. Se centra en el desarrollo y fabricación de equipos y maquinaria industrial, sistemas de automatización y control, soluciones para la generación y distribución de energía, así como tecnologías para la movilidad y la salud. La compañía juega un papel fundamental en diversos sectores, desde la fabricación hasta la infraestructura y la atención médica.

#### 3M [MMM]

Sub-Industria: Productos industriales / Bienes Diversificados.
Descripción: 3M es una empresa estadounidense diversificada que opera en múltiples industrias. Se destaca por su innovación en productos de consumo, bienes industriales, salud y seguridad, electrónica, entre otros. 3M es conocida por su amplia gama de productos, desde cintas adhesivas y productos de oficina hasta soluciones avanzadas en tecnologías industriales y de cuidado de la salud.

#### Lockheed Martin [LMT]

Sub-Industria: Aeroespacial y Defensa.
Descripción: Lockheed Martin es una empresa estadounidense líder en la industria aeroespacial y de defensa. Especializada en el diseño y fabricación de sistemas aeroespaciales, incluyendo aviones de combate, sistemas de misiles, tecnologías espaciales y equipos de defensa. Lockheed Martin es un contratista clave del gobierno de los Estados Unidos y está involucrado en proyectos aeroespaciales y de defensa a nivel mundial.


# Parte 2:

In [134]:
# Definir las acciones (tickers)
stocks = ["NSRGY", "WMT", "JNJ", "SIEGY", "MMM", "LMT"]
print(stocks)

#Definir las fechas de las observaciones:

end= datetime.datetime.now() #Que jale hasta hoy (los días* años)
start= end - datetime.timedelta(days=365*5)
print(end)
print(start)

['NSRGY', 'WMT', 'JNJ', 'SIEGY', 'MMM', 'LMT']
2023-11-22 22:47:05.812790
2018-11-23 22:47:05.812790


In [135]:
data= yf.download(stocks, start=start, end=end)["Adj Close"]
data.head()

[*********************100%%**********************]  6 of 6 completed


,JNJ,LMT,MMM,NSRGY,SIEGY,WMT
Date,,,,,,
2018-11-26,123.475273,258.371918,163.440094,76.606544,49.784264,87.409309
2018-11-27,125.091095,258.380707,164.118851,76.733231,49.579105,87.308258
2018-11-28,127.903481,262.449524,167.103714,76.959442,50.109089,89.531364
2018-11-29,127.388176,264.619476,167.283646,77.267097,49.878292,89.375214
2018-11-30,128.305252,264.804565,170.031357,77.113274,49.613304,89.705940


In [136]:
# Calcular razones de cambio: retornos % "Data pq así se llama mi  base"
returns= data.pct_change()
returns.head()

,JNJ,LMT,MMM,NSRGY,SIEGY,WMT
Date,,,,,,
2018-11-26,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-27,0.013086,0.000034,0.004153,0.001654,-0.004121,-0.001156
2018-11-28,0.022483,0.015747,0.018187,0.002948,0.010690,0.025463
2018-11-29,-0.004029,0.008268,0.001077,0.003998,-0.004606,-0.001744
2018-11-30,0.007199,0.000699,0.016425,-0.001991,-0.005313,0.003700


In [137]:
# Cambios medios por acción:
retorno_medio= returns.mean()
retorno_medio

JNJ      0.000243
LMT      0.000582
MMM     -0.000274
NSRGY    0.000378
SIEGY    0.000603
WMT      0.000553
dtype: float64

In [138]:
# definir pesos del portafolio: 
#Los pesos en el parentesis los pone en órden.
pesos= np.array([0.3, 0.2,0.2,0.1,0.1,0.1])
pesos

array([0.3, 0.2, 0.2, 0.1, 0.1, 0.1])

In [139]:
#Calcular el retorno promedio ponderado:
retorno_port=np.sum(retorno_medio*pesos)
retorno_port

0.0002880088465957751

In [140]:
#retorno acumulado por accion o cada una de las fechas:

#Crear la columna del retorno del portafolio:

returns["Portafolio"]=returns.dot(pesos)

#Calcular los retornos acumulados: Le sumaré 1 porque ahi inverí mi 100%

returns

ret_acumulado= (1+returns).cumprod()
ret_acumulado

,JNJ,LMT,MMM,NSRGY,SIEGY,WMT,Portafolio
Date,,,,,,,
2018-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-27,1.013086,1.000034,1.004153,1.001654,0.995879,0.998844,1.004401
2018-11-28,1.035863,1.015782,1.022416,1.004607,1.006525,1.024277,1.021919
2018-11-29,1.031690,1.024180,1.023517,1.008623,1.001889,1.022491,1.022354
2018-11-30,1.039117,1.024897,1.040328,1.006615,0.996566,1.026274,1.027695
...,...,...,...,...,...,...,...
2023-11-16,1.205971,1.724607,0.585597,1.447265,1.601309,1.785164,1.321418
2023-11-17,1.203480,1.721085,0.583333,1.453531,1.625614,1.777271,1.321031
2023-11-20,1.214089,1.717718,0.581130,1.444263,1.621998,1.776699,1.321832


In [141]:
# Ver el crecimiento final acumulado:
ret_acumulado.tail()

,JNJ,LMT,MMM,NSRGY,SIEGY,WMT,Portafolio
Date,,,,,,,
2023-11-16,1.205971,1.724607,0.585597,1.447265,1.601309,1.785164,1.321418
2023-11-17,1.203480,1.721085,0.583333,1.453531,1.625614,1.777271,1.321031
2023-11-20,1.214089,1.717718,0.581130,1.444263,1.621998,1.776699,1.321832
2023-11-21,1.223889,1.734515,0.575073,1.470501,1.621396,1.783105,1.327691
2023-11-22,1.221459,1.743223,0.583517,1.478203,1.632042,1.769491,1.332686


In [142]:
# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Diseño del dashboard
app.layout = html.Div([
    html.H1("Dashboard de Retornos Acumulados"),
    
    # Filtro de fecha
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=ret_acumulado.index.min(),
        end_date=ret_acumulado.index.max(),
        display_format='YYYY-MM-DD',
    ),
    
    # Gráfica de líneas
    dcc.Graph(id='line-plot'),
])

# Callback para actualizar la gráfica con el filtro de fecha
@app.callback(
    Output('line-plot', 'figure'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_graph(start_date, end_date):
    filtered_data = ret_acumulado.loc[start_date:end_date]
    fig = px.line(filtered_data, x=filtered_data.index, y=["JNJ", "LMT", "MMM", "SIEGY", "WMT", "NSRGY", "Portafolio"],
                  line_shape="linear")
    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)

# Parte 3:

In [17]:
# Definir las acciones para 3 años

stocks=["NSRGY", "WMT", "JNJ","SIEGY", "MMM","LMT"]
stocks

#Definir las fechas de las observaciones:

end= datetime.datetime.now()
start= end - datetime.timedelta(days=365*3)
print(end)
print(start)

2023-11-22 21:50:09.781313
2020-11-22 21:50:09.781313


In [18]:
data= yf.download(stocks, start=start, end=end)["Adj Close"]
data.head()

[*********************100%%**********************]  6 of 6 completed


,JNJ,LMT,MMM,NSRGY,SIEGY,WMT
Date,,,,,,
2020-11-23,132.713455,341.531525,152.686462,103.089325,61.154251,144.079117
2020-11-24,132.713455,348.558319,155.264267,103.469940,61.559067,144.489609
2020-11-25,132.538208,348.429291,155.299316,103.971222,61.945477,144.938293
2020-11-27,132.833405,344.745483,155.097672,104.630333,61.973076,144.718689
2020-11-30,133.460663,338.494965,151.450165,103.117180,61.430256,145.854691


In [19]:
returns=data.pct_change()
returns.head()

,JNJ,LMT,MMM,NSRGY,SIEGY,WMT
Date,,,,,,
2020-11-23,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-24,0.000000,0.020574,0.016883,0.003692,0.006620,0.002849
2020-11-25,-0.001320,-0.000370,0.000226,0.004845,0.006277,0.003105
2020-11-27,0.002227,-0.010573,-0.001298,0.006339,0.000446,-0.001515
2020-11-30,0.004722,-0.018131,-0.023517,-0.014462,-0.008759,0.007850


In [20]:
#FUNCIÓN DE ANALISIS 

def print_statistics(data):
    mean=data.mean()*100
    std=data.std()*100
    skewness=data.skew()
    kurtosis=data.kurtosis()
    
    print("Mean: ",mean)
    print("Std: ",std)
    print("skewness: ",skewness)
    print("kurtosis: ",kurtosis)

In [21]:
retorno_medio= returns.mean()
retorno_medio

JNJ      0.000222
LMT      0.000462
MMM     -0.000505
NSRGY    0.000185
SIEGY    0.000572
WMT      0.000176
dtype: float64

### Acciones de Jhonson & Jhonson

In [22]:
### Retorno medio

retorno_medio_JNJ = retorno_medio.iloc[0]
print("El retorno medio de esta accion es de:", retorno_medio_JNJ)
print("")
### PRUEBA DEL AUTOMATIZADO
stats_JNJ = pf.timeseries.perf_stats(returns["JNJ"])

# Retorno anual, valatilidad y sharpe

RVS_JNJ = stats_JNJ.iloc[0:8]
print(RVS_JNJ)

# Sesgo y Kurtosis

print("Sesgo y Kurtosis de JNJ")
print("")
print(print_statistics(returns["JNJ"]))

El retorno medio de esta accion es de: 0.00022190019352781096

Annual return         0.043612
Cumulative returns    0.136433
Annual volatility     0.162543
Sharpe ratio          0.344025
Calmar ratio          0.236873
Stability             0.128693
Max drawdown         -0.184117
Omega ratio           1.060501
dtype: float64
Sesgo y Kurtosis de JNJ

Mean:  0.022190019352781098
Std:  1.0239241127688234
skewness:  0.2556772975752334
kurtosis:  3.1904173560167
None


C:\Users\Marco\anaconda3\Lib\site-packages\pyfolio\timeseries.py:724: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



### Acciones de Lockhead Martin

In [23]:
### Retorno medio

retorno_medio_LMT = retorno_medio.iloc[1]
print("El retorno medio de esta accion es de:", retorno_medio_LMT)
print("")
# PRUEBA DEL AUTOMATIZADO
stats_LMT = pf.timeseries.perf_stats(returns["LMT"])

# Retorno anual, volatilidad y Sharpe
RVS_LMT = stats_LMT.iloc[0:8]
print(RVS_LMT)

# Sesgo y Kurtosis
print("Sesgo y Kurtosis de LMT")
print("")
print(print_statistics(returns["LMT"]))

El retorno medio de esta accion es de: 0.000461844568258179

Annual return         0.096753
Cumulative returns    0.318766
Annual volatility     0.218326
Sharpe ratio          0.533079
Calmar ratio          0.491467
Stability             0.732434
Max drawdown         -0.196866
Omega ratio           1.104342
dtype: float64
Sesgo y Kurtosis de LMT

Mean:  0.0461844568258179
Std:  1.3753236515615854
skewness:  -0.30175061919749807
kurtosis:  12.722696586903316
None


C:\Users\Marco\anaconda3\Lib\site-packages\pyfolio\timeseries.py:724: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



### Acciones de 3M

In [24]:
### Retorno medio

retorno_medio_MMM = retorno_medio.iloc[2]
print("El retorno medio de esta accion es de:", retorno_medio_MMM)
print("")
# PRUEBA DEL AUTOMATIZADO
stats_MMM = pf.timeseries.perf_stats(returns["MMM"])

# Retorno anual, volatilidad y Sharpe
RVS_MMM = stats_MMM.iloc[0:8]
print(RVS_MMM)

# Sesgo y Kurtosis
print("Sesgo y Kurtosis de MMM")
print("")
print(print_statistics(returns["MMM"]))

El retorno medio de esta accion es de: -0.0005049738987138481

Annual return        -0.145366
Cumulative returns   -0.375387
Annual volatility     0.244933
Sharpe ratio         -0.519545
Calmar ratio         -0.268834
Stability             0.861537
Max drawdown         -0.540728
Omega ratio           0.911903
dtype: float64
Sesgo y Kurtosis de MMM

Mean:  -0.05049738987138481
Std:  1.54293024068556
skewness:  0.027726335759597705
kurtosis:  4.486077847673085
None


C:\Users\Marco\anaconda3\Lib\site-packages\pyfolio\timeseries.py:724: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



### Acciones de Nestle

In [25]:
### Retorno medio

retorno_medio_NSRGY = retorno_medio.iloc[3]
print("El retorno medio de esta accion es de:", retorno_medio_NSRGY)
print("")
# PRUEBA DEL AUTOMATIZADO
stats_NSRGY = pf.timeseries.perf_stats(returns["NSRGY"])

# Retorno anual, volatilidad y Sharpe
RVS_NSRGY = stats_NSRGY.iloc[0:8]
print(RVS_NSRGY)

# Sesgo y Kurtosis
print("Sesgo y Kurtosis de NSRGY")
print("")
print(print_statistics(returns["NSRGY"]))

El retorno medio de esta accion es de: 0.000184755931076469

Annual return         0.031842
Cumulative returns    0.098465
Annual volatility     0.174197
Sharpe ratio          0.267275
Calmar ratio          0.130856
Stability             0.051117
Max drawdown         -0.243340
Omega ratio           1.045292
dtype: float64
Sesgo y Kurtosis de NSRGY

Mean:  0.0184755931076469
Std:  1.0973371004607324
skewness:  -0.1479658826918738
kurtosis:  1.0936579858851077
None


C:\Users\Marco\anaconda3\Lib\site-packages\pyfolio\timeseries.py:724: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



### Acciones de Siemmens

In [26]:
### Retorno medio

retorno_medio_SIEGY = retorno_medio.iloc[4]
print("El retorno medio de esta accion es de:", retorno_medio_SIEGY)
print("")
# PRUEBA DEL AUTOMATIZADO
stats_SIEGY = pf.timeseries.perf_stats(returns["SIEGY"])

# Retorno anual, volatilidad y Sharpe
RVS_SIEGY = stats_SIEGY.iloc[0:8]
print(RVS_SIEGY)

# Sesgo y Kurtosis
print("Sesgo y Kurtosis de SIEGY")
print("")
print(print_statistics(returns["SIEGY"]))

El retorno medio de esta accion es de: 0.0005722002704784653

Annual return         0.099478
Cumulative returns    0.328608
Annual volatility     0.314321
Sharpe ratio          0.458749
Calmar ratio          0.218122
Stability             0.002728
Max drawdown         -0.456067
Omega ratio           1.079678
dtype: float64
Sesgo y Kurtosis de SIEGY

Mean:  0.057220027047846525
Std:  1.9800351595943424
skewness:  0.20049031077029628
kurtosis:  1.7143573713471816
None


C:\Users\Marco\anaconda3\Lib\site-packages\pyfolio\timeseries.py:724: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



### Acciones de Walmart

In [27]:
### Retorno medio

retorno_medio_WMT = retorno_medio.iloc[5]
print("El retorno medio de esta accion es de:", retorno_medio_WMT)
print("")

stats_WMT = pf.timeseries.perf_stats(returns["WMT"])

# Retorno anual, volatilidad y Sharpe
RVS_WMT = stats_WMT.iloc[0:8]
print(RVS_WMT)

# Sesgo y Kurtosis
print("Sesgo y Kurtosis de WMT")
print("")
print(print_statistics(returns["WMT"]))

El retorno medio de esta accion es de: 0.00017611339328506645

Annual return         0.023958
Cumulative returns    0.073507
Annual volatility     0.201925
Sharpe ratio          0.219787
Calmar ratio          0.093076
Stability             0.317844
Max drawdown         -0.257397
Omega ratio           1.041936
dtype: float64
Sesgo y Kurtosis de WMT

Mean:  0.017611339328506646
Std:  1.2720077967240948
skewness:  -1.667690544145023
kurtosis:  14.511968750788593
None


C:\Users\Marco\anaconda3\Lib\site-packages\pyfolio\timeseries.py:724: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



#### Argumentación de las acciones que se trabajarán:

##### Lockheed Martin (LMT):
Razones para la selección: Lockheed Martin muestra un rendimiento sólido con un retorno medio más alto y una volatilidad anual moderada. El Sharpe ratio y Calmar ratio son significativamente superiores en comparación con otras acciones. Además, la skewness y kurtosis son relativamente bajas, indicando una distribución más normal de los rendimientos. Además de esto, se cree que el gobierno de EEUU que es su principal cliente se preparará para elevar el gasto de defensa, lo que significaría un aumento de ventas.

##### Siemens AG (SIEGY):
Razones para la selección: Siemens también tiene un rendimiento atractivo con un retorno medio sólido y una volatilidad anual moderada. El Sharpe ratio es significativamente positivo, lo que indica un buen rendimiento ajustado al riesgo. Aunque la skewness y kurtosis no son perfectas, son aceptables en comparación con algunas otras acciones. Además de esto hay que considerar su presencia en el desarrollo en teconologías para vehículos eléctricos y software de IA.

##### Johnson & Johnson (JNJ):
Razones para la selección: Aunque J&J tiene un retorno medio más bajo en comparación con LMT y SIEGY, muestra una volatilidad anual más baja y un Sharpe ratio positivo. Es una opción más conservadora, adecuada si nuestros clientes buscan estabilidad y poco riesgo. Esto permitirá de que las personas tengan una acción que sepan que es sólida. 

##### Nestle SA (NSRGY):
Razones para la selección: Aunque NSRGY tiene un retorno medio más bajo, su volatilidad es relativamente baja, y el Sharpe ratio es positivo. Además, la skewness y kurtosis son aceptables. Esta acción puede ser considera como un contrapeso, ya que tiene una estabilidad bastante buena y el riesgo es poco. Por lo que puede llegar a ser una buena aportación al portafolio ya que este tipo de industrias siempre tienen una demanda estable y cíclica.


# Parte 4:

In [28]:
#PORTAFOLIO CON LAS ACCIONES RECOMENDADAS
stocks2=["LMT","SIEGY","JNJ", "NSRGY"]
stocks2

end= datetime.datetime.now() #Que jale hasta hoy (los días* años)
start= end - datetime.timedelta(days=365*3)
print(end)
print(start)

2023-11-22 21:50:38.572980
2020-11-22 21:50:38.572980


In [29]:
#Datos con ticker grabados en lista de stocks por 3 años
data2= yf.download(stocks2, start=start, end=end)["Adj Close"].dropna()
data2.head()

[*********************100%%**********************]  4 of 4 completed


,JNJ,LMT,NSRGY,SIEGY
Date,,,,
2020-11-23,132.713486,341.531494,103.089333,61.154251
2020-11-24,132.713486,348.558289,103.469933,61.559067
2020-11-25,132.538223,348.429352,103.971222,61.945477
2020-11-27,132.833389,344.745575,104.630333,61.973076
2020-11-30,133.460693,338.495056,103.117180,61.430256


In [30]:
#Razones de cambio: retornos %
returns2= data2.pct_change()
returns2.head()

,JNJ,LMT,NSRGY,SIEGY
Date,,,,
2020-11-23,NaN,NaN,NaN,NaN
2020-11-24,0.000000,0.020574,0.003692,0.006620
2020-11-25,-0.001321,-0.000370,0.004845,0.006277
2020-11-27,0.002227,-0.010573,0.006339,0.000446
2020-11-30,0.004722,-0.018131,-0.014462,-0.008759


In [31]:
#Cambios medios por acción:
retorno_medio2= returns2.mean()
retorno_medio2

JNJ      0.000222
LMT      0.000462
NSRGY    0.000185
SIEGY    0.000572
dtype: float64

In [32]:
#DEFINIR PESOS DEL PORTAFOLIO
pesos= np.array([0.25,0.25,0.25,0.25])
pesos

array([0.25, 0.25, 0.25, 0.25])

In [33]:
#Calcular el retorno promedio ponderado:
retorno_port=np.sum(retorno_medio2*pesos)
retorno_port

0.0003601751442721183

In [34]:
###retorno acumulado por accion o cada una de las fechas:
#RETORNO DEL PORTAFOLIO / HISTORICO
returns2["Portafolio"]=returns2.dot(pesos)

#Calcular los retornos acumulados: Le sumaré 1 porque ahi invertí mi 100%
returns2

ret_acumulado= (1+returns2).cumprod()
ret_acumulado

,JNJ,LMT,NSRGY,SIEGY,Portafolio
Date,,,,,
2020-11-23,NaN,NaN,NaN,NaN,NaN
2020-11-24,1.000000,1.020574,1.003692,1.006620,1.007721
2020-11-25,0.998679,1.020197,1.008555,1.012938,1.010098
2020-11-27,1.000903,1.009411,1.014948,1.013389,1.009703
2020-11-30,1.005630,0.991109,1.000270,1.004513,1.000457
...,...,...,...,...,...
2023-11-16,1.122023,1.304682,1.075475,1.303589,1.255376
2023-11-17,1.119705,1.302018,1.080131,1.323375,1.260209
2023-11-20,1.129576,1.299470,1.073244,1.320432,1.259661


In [35]:
#comparar portafolio con el S&P
syp=yf.download("^GSPC", start=start, end=end)["Adj Close"]
syp

[*********************100%%**********************]  1 of 1 completed


Date
2020-11-23    3577.590088
2020-11-24    3635.409912
2020-11-25    3629.649902
2020-11-27    3638.350098
2020-11-30    3621.629883
                 ...     
2023-11-16    4508.240234
2023-11-17    4514.020020
2023-11-20    4547.379883
2023-11-21    4538.189941
2023-11-22    4556.620117
Name: Adj Close, Length: 755, dtype: float64

In [36]:
#retorno en porcentajes
retsyp=syp.pct_change()
retsyp

Date
2020-11-23         NaN
2020-11-24    0.016162
2020-11-25   -0.001584
2020-11-27    0.002397
2020-11-30   -0.004596
                ...   
2023-11-16    0.001190
2023-11-17    0.001282
2023-11-20    0.007390
2023-11-21   -0.002021
2023-11-22    0.004061
Name: Adj Close, Length: 755, dtype: float64

In [37]:
#retorno medio:
retsyp_total=np.mean(retsyp)
retsyp_total

0.00038234266782819827

In [38]:
#RETORNO ACTIVO
retact= retorno_port-retsyp_total
retact

-2.2167523556079966e-05

# Parte 5:

In [50]:
sigma = risk_models.sample_cov(data2)
print(sigma)

            JNJ       LMT     NSRGY     SIEGY
JNJ    0.026420  0.008865  0.006888  0.008258
LMT    0.008865  0.047666  0.005841  0.001172
NSRGY  0.006888  0.005841  0.030345  0.018033
SIEGY  0.008258  0.001172  0.018033  0.098798


In [51]:
#Retorno
mu1 = expected_returns.mean_historical_return(data2)
print(mu1)

JNJ      0.043671
LMT      0.096888
NSRGY    0.031885
SIEGY    0.099617
dtype: float64


In [52]:
#Frontera eficiente
ef1 = EfficientFrontier(mu1,sigma)

In [53]:
#Máximo portafolio con el sharpe

weights= ef1.max_sharpe()
print(weights)

OrderedDict([('JNJ', 0.0510328003932084), ('LMT', 0.6349513881269807), ('NSRGY', 0.0), ('SIEGY', 0.314015811479811)])


In [55]:
#Resultados del portafolio de máximo sharpe
ef1.portfolio_performance(verbose=True)

Expected annual return: 9.5%
Annual volatility: 17.4%
Sharpe Ratio: 0.43


(0.09502880862589153, 0.174168217601037, 0.43078358186887017)

In [57]:
#Portafolio que la minima volatilidad

ef2 = EfficientFrontier(mu1,sigma) #Limpiar los resultados de la última optimización o volver
#a correr la frontera antes de hacer una nueva optimización
weights2 = ef2.min_volatility()
print(weights2)

OrderedDict([('JNJ', 0.4033156628193596), ('LMT', 0.2019080255857481), ('NSRGY', 0.3383691111993144), ('SIEGY', 0.0564072003955779)])


In [59]:
#Resultados de portafolio que la mínima volatilidad

ef2.portfolio_performance(verbose=True)

Expected annual return: 5.4%
Annual volatility: 12.3%
Sharpe Ratio: 0.27


(0.05358381497907703, 0.12345836391229675, 0.27202543363473164)

# Parte 6:

#### a.	¿Hay diferencia entre ambos portafolios? 

Tenemos de que en primera instancia sí que hay diferencias en el rendimientos, después de todo lo que nuesrtos clientes quieren es que tengan un redimiento mayor. El portafolio con el máximo sharpe tiene un mejor rendimiento. Lo mismo sucede con el sharpe del portafolio con el máximo de sharpe, es una un balance. En resumen, el Portafolio con máximo de sharpe tiene un rendimiento esperado y un Sharpe Ratio más altos, pero también tiene una mayor volatilidad en comparación con el Portafolio 2. La elección entre los dos dependerá de las preferencias del inversionista en términos de tolerancia al riesgo y objetivos de rendimiento.

#### b.	¿Hay diferencia con el portafolio del inciso 4? 

Si tiene una diferencia, esto más que todo en cuestiones de rentabilidad. Pero esto se debe a que el portafolio del inciso 6 ya está optimizado.

#### c.	¿En cuál recomendaría invertir? 

Recomiendo el portafolio con el máximo sharpe ya que tiene un buen balance entre riesgo y rendimiento. Además que el rendimiento es mayor y nuestros clientes eso es lo que están buscando.


# Parte 7:

In [104]:
#PORTAFOLIO CON LAS ACCIONES RECOMENDADAS
stocks_MS=["LMT","SIEGY","JNJ", "NSRGY"]
stocks_MS

['LMT', 'SIEGY', 'JNJ', 'NSRGY']

In [105]:
data3= yf.download(stocks_MS, start=start, end=end)["Adj Close"].dropna()
data3.head()

[*********************100%%**********************]  4 of 4 completed


,JNJ,LMT,NSRGY,SIEGY
Date,,,,
2020-11-23,132.713455,341.531464,103.089325,61.154251
2020-11-24,132.713455,348.558258,103.469940,61.559067
2020-11-25,132.538193,348.429352,103.971222,61.945477
2020-11-27,132.833374,344.745514,104.630333,61.973076
2020-11-30,133.460663,338.494995,103.117180,61.430256


In [119]:
returns3= data3.pct_change()
returns3.head()

,JNJ,LMT,NSRGY,SIEGY
Date,,,,
2020-11-23,NaN,NaN,NaN,NaN
2020-11-24,0.000000,0.020574,0.003692,0.006620
2020-11-25,-0.001321,-0.000370,0.004845,0.006277
2020-11-27,0.002227,-0.010573,0.006339,0.000446
2020-11-30,0.004722,-0.018131,-0.014462,-0.008759


In [154]:
pesos3 = np.array([0.0510328003932084, 0.6349513881269807, 0, 0.314015811479811])
returns3["Portafolio_SHARPE"] = returns3.dot(pesos3)
ret_acumulado_sharpe = (1 + returns3["Portafolio_SHARPE"]).cumprod().dropna()
ret_acumulado_sharpe.head()

ValueError: Dot product shape mismatch, (755, 5) vs (4,)

In [121]:
### PORTAFOLIO 2 CON ACCIONES RECOMENDADAS

#PORTAFOLIO CON LAS ACCIONES RECOMENDADAS
stocks_MS2=["LMT","SIEGY","JNJ", "NSRGY"]
stocks_MS2

['LMT', 'SIEGY', 'JNJ', 'NSRGY']

In [122]:
data4= yf.download(stocks_MS2, start=start, end=end)["Adj Close"].dropna()
data4.head()

[*********************100%%**********************]  4 of 4 completed


,JNJ,LMT,NSRGY,SIEGY
Date,,,,
2020-11-23,132.713501,341.531494,103.089325,61.154251
2020-11-24,132.713501,348.558289,103.469933,61.559063
2020-11-25,132.538223,348.429321,103.971230,61.945477
2020-11-27,132.833405,344.745575,104.630333,61.973072
2020-11-30,133.460648,338.495056,103.117180,61.430252


In [123]:
returns4= data3.pct_change()
returns4.head()

,JNJ,LMT,NSRGY,SIEGY
Date,,,,
2020-11-23,NaN,NaN,NaN,NaN
2020-11-24,0.000000,0.020574,0.003692,0.006620
2020-11-25,-0.001321,-0.000370,0.004845,0.006277
2020-11-27,0.002227,-0.010573,0.006339,0.000446
2020-11-30,0.004722,-0.018131,-0.014462,-0.008759


In [124]:
#Pesos Portafolio con menos volatilidad
pesos4= np.array([0.4033156628193596,0.2019080255857481,0.3383691111993144,0.0564072003955779])

Volatilidad = returns4["Portafolio_VOLAT"]=returns4.dot(pesos4)
Volatilidad

#Calcular los retornos acumulados: Le sumaré 1 porque ahi invertí mi 100%
returns4

ret_acumuladoVolatilidad= (1+returns4).cumprod().dropna()
ret_acumuladoVolatilidad.head()

,JNJ,LMT,NSRGY,SIEGY,Portafolio_VOLAT
Date,,,,,
2020-11-24,1.000000,1.020574,1.003692,1.006620,1.005777
2020-11-25,0.998679,1.020197,1.008555,1.012938,1.007171
2020-11-27,1.000904,1.009411,1.014948,1.013389,1.008111
2020-11-30,1.005630,0.991109,1.000270,1.004513,1.000910
2020-12-01,1.024884,0.991082,1.007384,1.026027,1.012251


In [146]:
# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Diseño del dashboard
app.layout = html.Div([
    html.H1("Dashboard de Retornos Acumulados"),
    
    # Dropdown para seleccionar el dataset
    dcc.Dropdown(
        id='dataset-selector',
        options=[
            {'label': 'Volatilidad', 'value': 'volatilidad'},
            {'label': 'Sharpe', 'value': 'sharpe'}
        ],
        value='volatilidad',
        style={'width': '50%'}
    ),
    
    # Filtro de fecha
    dcc.DatePickerRange(
        id='date-picker-range',
        display_format='YYYY-MM-DD',
    ),
    
    # Gráfica de líneas
    dcc.Graph(id='line-plot'),
])

# Callback para actualizar la gráfica con el filtro de fecha y el dataset seleccionado
@app.callback(
    Output('line-plot', 'figure'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('dataset-selector', 'value')]
)
def update_graph(start_date, end_date, selected_dataset):
    if selected_dataset == 'volatilidad':
        filtered_data = ret_acumuladoVolatilidad.loc[start_date:end_date]
        y_columns = ["JNJ", "LMT", "NSRGY", "SIEGY", "Portafolio_VOLAT"]
    elif selected_dataset == 'sharpe':
        filtered_data = ret_acumulado_sharpe.loc[start_date:end_date]
        y_columns = ["JNJ", "LMT", "NSRGY", "SIEGY", "Portafolio_SHARPE"]
    else:
        # Manejar un caso por defecto o lanzar una excepción según tus necesidades
        return px.line()

    fig = px.line(filtered_data, x=filtered_data.index, y=y_columns, line_shape="linear")
    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)

In [152]:

ret_acumulado_sharpe
ret_acumuladoVolatilidad

,JNJ,LMT,NSRGY,SIEGY,Portafolio_VOLAT
Date,,,,,
2020-11-24,1.000000,1.020574,1.003692,1.006620,1.005777
2020-11-25,0.998679,1.020197,1.008555,1.012938,1.007171
2020-11-27,1.000904,1.009411,1.014948,1.013389,1.008111
2020-11-30,1.005630,0.991109,1.000270,1.004513,1.000910
2020-12-01,1.024884,0.991082,1.007384,1.026027,1.012251
...,...,...,...,...,...
2023-11-16,1.122023,1.304682,1.075475,1.303589,1.186344
2023-11-17,1.119706,1.302018,1.080131,1.323375,1.187621
2023-11-20,1.129577,1.299470,1.073244,1.320432,1.188663
